# Simple Gradio app to demo GPPR vs DBO

### Import Statement

In [7]:
import random
import pandas as pd
import gradio as gr
import matplotlib.pyplot as plt

%run "Finance_Management_GPPR.ipynb"
%run "Finance_Management_DBO.ipynb"

=== GPPR Budget Result ===
Category        Spend s[i]   Min m[i]   Priority  Budget b[i]    Cut (s-b)
--------------------------------------------------------------------------
Rent               1200.00    1200.00          1      1200.00         0.00
Groceries           400.00     300.00          2       300.00       100.00
Restaurants         350.00     100.00          4       214.53       135.47
Transport           150.00     100.00          3       113.87        36.13
Entertainment       200.00      50.00          5       126.60        73.40

Diagnostics:
           algorithm: GPPR
 discretionary_total: 550.0
feasible_full_target: True
           ops_count: 67
adjustment_iterations: 1
          runtime_ms: 0.0
         total_spend: 2300.0
        total_budget: 1955.0
       target_budget: 1955.0
        required_cut: 345.0
          actual_cut: 345.0
        target_error: 0.0
              min_ok: True
      target_reached: True
=== DBO Budget Result ===
Category        Spend s[i] 

### Helper Functions

In [3]:
def parse_float_list(s: str):
    """
    Parse a comma-separated string into a list of floats.
    Example: "1200, 400, 350" -> [1200.0, 400.0, 350.0]
    """
    if not s.strip():
        return []
    return [float(x.strip()) for x in s.split(",") if x.strip()]

def parse_int_list(s: str):
    """
    Parse a comma-separated string into a list of ints.
    Example: "1, 2, 4" -> [1, 2, 4]
    """
    if not s.strip():
        return []
    return [int(x.strip()) for x in s.split(",") if x.strip()]

def parse_str_list(s: str):
    """
    Parse a comma-separated string into a list of strings.
    Example: "Rent, Groceries" -> ["Rent", "Groceries"]
    """
    if not s.strip():
        return []
    return [x.strip() for x in s.split(",") if x.strip()]

In [8]:
# Default example (same spirit as earlier notebooks)
default_data = {
    "Category": ["Rent", "Groceries", "Restaurants", "Transport", "Entertainment"],
    "Spend s[i]": [1200.0, 400.0, 350.0, 150.0, 200.0],
    "Min m[i]": [1200.0, 300.0, 100.0, 100.0, 50.0],
    "Priority p[i]": [1, 2, 4, 3, 5],
}
default_df = pd.DataFrame(default_data)

def generate_random_example(num_categories: int = 6) -> pd.DataFrame:
    """
    Generate a random but reasonable budgeting example.
    Higher-priority categories tend to have higher spends.
    """
    categories = [f"Cat_{i+1}" for i in range(num_categories)]
    spends = []
    mins = []
    priorities = []

    for i in range(num_categories):
        base = random.randint(80, 600)  # random baseline spend
        spend = float(base)
        # minimum is between 40% and 90% of spend
        min_budget = float(random.randint(int(0.4 * base), int(0.9 * base)))
        # priority 1–5
        priority = random.randint(1, 5)

        categories[i] = categories[i]
        spends.append(spend)
        mins.append(min_budget)
        priorities.append(priority)

    df = pd.DataFrame(
        {
            "Category": categories,
            "Spend s[i]": spends,
            "Min m[i]": mins,
            "Priority p[i]": priorities,
        }
    )
    return df

### Gradio Skeleton

In [9]:
def run_budget_ui(
    algorithm: str,
    table: pd.DataFrame,
    r: float,
    unit: float,
):
    """
    Gradio UI core:
    - validates the DataFrame
    - calls GPPR/DBO
    - returns per-category results, textual summary, and a bar chart
    """

    # -------- 1) Validate table structure --------
    required_cols = ["Category", "Spend s[i]", "Min m[i]", "Priority p[i]"]
    if table is None or table.empty:
        df_error = pd.DataFrame({"Error": ["Input table is empty. Add at least one row."]})
        return df_error, "Error: no rows in the table.", plt.figure()

    for col in required_cols:
        if col not in table.columns:
            df_error = pd.DataFrame({"Error": [f"Missing required column: {col}"]})
            return df_error, f"Error: missing column '{col}'.", plt.figure()

    # Drop rows that are completely empty
    table = table.dropna(how="all")
    if table.empty:
        df_error = pd.DataFrame({"Error": ["All rows are empty. Fill in at least one row."]})
        return df_error, "Error: all rows are empty.", plt.figure()

    warnings = []

    # -------- 2) Extract lists and basic checks --------
    try:
        categories = table["Category"].fillna("").tolist()
        spends = table["Spend s[i]"].astype(float).tolist()
        mins = table["Min m[i]"].astype(float).tolist()
        priorities = table["Priority p[i]"].astype(int).tolist()
    except Exception as e:
        df_error = pd.DataFrame({"Error": [f"Could not parse numbers: {e}"]})
        return df_error, "Error parsing numeric values. Check that spends, mins, and priorities are valid numbers.", plt.figure()

    n = len(spends)
    if n == 0:
        df_error = pd.DataFrame({"Error": ["No valid rows after parsing."]})
        return df_error, "Error: no valid rows.", plt.figure()

    # Fill missing/blank categories with C1, C2, ...
    for i in range(n):
        if not str(categories[i]).strip():
            categories[i] = f"C{i+1}"

    # Ensure positive unit
    if unit is None or unit <= 0:
        unit = 10.0
        warnings.append("Unit was non-positive or missing; defaulting to 10.0.")

    # Soft validation for m[i] > s[i]
    for i in range(n):
        if mins[i] > spends[i]:
            warnings.append(
                f"Warning: For category '{categories[i]}', Min m[i] ({mins[i]:.2f}) > Spend s[i] ({spends[i]:.2f})."
            )

    # -------- 3) Run algorithms --------
    rows = []
    summaries = []
    budget_by_algo = {}  # for plotting

    def add_result(algo_name, b, diag):
        # Store budgets for plotting
        budget_by_algo[algo_name] = b

        # Add table rows
        for i in range(n):
            old_spend = spends[i]
            new_budget = b[i]
            cut = old_spend - new_budget
            cut_pct = (cut / old_spend * 100.0) if old_spend > 0 else 0.0

            rows.append(
                {
                    "Algorithm": algo_name,
                    "Category": categories[i],
                    "Spend s[i]": old_spend,
                    "Min m[i]": mins[i],
                    "Priority p[i]": priorities[i],
                    "Budget b[i]": new_budget,
                    "Cut (s-b)": cut,
                    "Cut %": cut_pct,
                }
            )

        # Summaries
        summary_lines = [
            f"--- {algo_name} ---",
            f"Total spend:      {diag['total_spend']:.2f}",
            f"Target budget:    {diag['target_budget']:.2f}",
            f"Actual budget:    {diag['total_budget']:.2f}",
            f"Required cut:     {diag['required_cut']:.2f}",
            f"Actual cut:       {diag['actual_cut']:.2f}",
            f"Target error:     {diag['target_error']:.2f}",
            f"Min constraints OK: {diag['min_ok']}",
            f"Target reached:     {diag['target_reached']}",
        ]
        if "runtime_ms" in diag and diag["runtime_ms"] is not None:
            summary_lines.append(f"Runtime (ms):       {diag['runtime_ms']:.3f}")

        summaries.append("\n".join(summary_lines))

    try:
        if algorithm in ("GPPR", "Both"):
            b_gppr, diag_gppr = gppr_budget(spends, mins, priorities, r, adjustment_unit=unit)
            add_result("GPPR", b_gppr, diag_gppr)

        if algorithm in ("DBO", "Both"):
            b_dbo, diag_dbo = dbo_budget(spends, mins, priorities, r, unit=unit)
            add_result("DBO", b_dbo, diag_dbo)

    except Exception as e:
        df_error = pd.DataFrame({"Error": [f"Algorithm error: {e}"]})
        return df_error, "Error while running algorithms. Check inputs.", plt.figure()

    if not rows:
        df_error = pd.DataFrame({"Error": ["No algorithm was executed."]})
        return df_error, "Choose GPPR, DBO, or Both.", plt.figure()

    results_df = pd.DataFrame(rows)

    # -------- 4) Build bar chart (spend vs budgets) --------
    fig, ax = plt.subplots(figsize=(8, 4))
    x = range(n)
    ax.bar(x, spends, width=0.25, label="Original spend")

    if "GPPR" in budget_by_algo:
        ax.bar(
            [xi + 0.25 for xi in x],
            budget_by_algo["GPPR"],
            width=0.25,
            label="GPPR budget",
        )
    if "DBO" in budget_by_algo:
        shift = 0.5 if "GPPR" in budget_by_algo else 0.25
        ax.bar(
            [xi + shift for xi in x],
            budget_by_algo["DBO"],
            width=0.25,
            label="DBO budget",
        )

    ax.set_xticks([xi + 0.25 for xi in x])
    ax.set_xticklabels(categories, rotation=30, ha="right")
    ax.set_ylabel("Amount")
    ax.set_title("Original spends vs algorithm budgets")
    ax.legend()
    plt.tight_layout()

    # -------- 5) Final summary text --------
    all_summaries = []
    if warnings:
        all_summaries.append("WARNINGS:")
        all_summaries.extend(warnings)
        all_summaries.append("")
    all_summaries.extend(summaries)
    summary_text = "\n".join(all_summaries)

    return results_df, summary_text, fig

### Gradio Implementation

In [10]:
default_r = 0.15
default_unit = 10.0

with gr.Blocks(title="Budget Allocation – GPPR vs DBO") as demo:
    gr.Markdown("# Monthly Budget Planner\nCompare GPPR and DBO on your own data.")

    with gr.Row():
        algorithm = gr.Radio(
            ["GPPR", "DBO", "Both"],
            value="Both",
            label="Algorithm",
            info="Choose which algorithm to run on the table below.",
        )
        r_input = gr.Slider(
            minimum=0.0,
            maximum=0.5,
            value=default_r,
            step=0.01,
            label="Savings rate r",
            info="Fraction of total spend you want to cut (e.g., 0.15 = 15% savings).",
        )
        unit_input = gr.Number(
            value=default_unit,
            label="Unit (currency units)",
            info="Step size for cuts (used by both GPPR adjustment and DBO DP).",
        )

    gr.Markdown(
        "### Categories and budgets\n"
        "Edit the table below or use the buttons to load an example."
    )

    table_input = gr.Dataframe(
        value=default_df,
        headers=["Category", "Spend s[i]", "Min m[i]", "Priority p[i]"],
        datatype=["str", "number", "number", "number"],
        row_count=(5, "dynamic"),
        col_count=(4, "fixed"),
        label="Budget table",
        type="pandas",
        interactive=True,
    )

    with gr.Row():
        default_button = gr.Button("Use default example")
        random_button = gr.Button("Random example")
        run_button = gr.Button("Run budget allocation", variant="primary")

    output_table = gr.Dataframe(
        label="Per-category results",
        interactive=False,
    )
    output_summary = gr.Textbox(
        label="Summary",
        lines=12,
    )
    output_plot = gr.Plot(
        label="Original vs new budgets",
    )

    # --- Button wiring ---

    # Default example button → reset table, r, and unit
    def load_default():
        return default_df, default_r, default_unit

    default_button.click(
        fn=load_default,
        inputs=None,
        outputs=[table_input, r_input, unit_input],
    )

    # Random example button → randomize table (keep r, unit unchanged)
    def load_random():
        return generate_random_example()

    random_button.click(
        fn=load_random,
        inputs=None,
        outputs=[table_input],
    )

    # Run button → core algorithm call
    run_button.click(
        fn=run_budget_ui,
        inputs=[algorithm, table_input, r_input, unit_input],
        outputs=[output_table, output_summary, output_plot],
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
